# Managing assignment files manually

Distributing assignments to students and collecting them can be a logistical nightmare. If you are relying on distributing the release version of the assignment to the students using your institution's existing learning management system the process of downloading the students submissions from the learning management system and then getting them back into ``nbgrader`` can be simplified by relying on some of nbgrader's built-in functionality.

## Releasing and fetching assignments

After an assignment has been created using `nbgrader assign`, the instructor must actually release that assignment to students. This section of the documentation assumes you are using your institution's existing learning management system for distributing the release version of the assignment. It is also assumed that the students will fetch the assignments from - and submit their assignments to - the learning management system.

## Collecting assignments

For demo purposes we have already created the directories needed by the ``nbgrader zip_collect`` sub-command and placed the downloaded assignment submission files and archive (zip) files in there. For example we have one ``.zip`` file and one ``.ipynb`` file:

In [1]:
%%bash

ls -l downloaded/ps1/archive

total 32
-rw-rw-r-- 1 logan logan 18022 Jan 20 16:11 gradebook_xyz101_s1_2016_ps1_2016-02-10-17-30-10.zip
-rw-rw-r-- 1 logan logan  8830 Jan 20 16:46 ps1_hacker_attempt_2016-01-30-20-30-10_problem1.ipynb


But before we can run the ``nbgrader zip_collect`` sub-command we first need to specify a few config options:

In [2]:
%%file nbgrader_config.py

c = get_config()

# only set for demo purposes so as to not mess up the other documentation
c.ZipCollectApp.submitted_directory = 'submitted_zip'

c.ZipCollectApp.strict = True
c.ZipCollectApp.auto_update_database = True

c.FileNameCollectorPlugin.named_regexp = (
    '.*_(?P<student_id>\w+)_attempt_'
    '(?P<timestamp>[0-9\-]+)_'
    '(?P<file_id>\w+)'
)

Overwriting nbgrader_config.py


Setting the ``strict`` flag ``True`` skips any submitted notebooks with invalid names.

Setting the ``auto_update_database`` flag ``True`` will automatically update the database by adding missing student entries and update existing student entries with updated ``first_name``, ``last_name``, and ``email`` data.

By default the ``nbgrader zip_collect`` sub-command uses the ``FileNameCollectorPlugin`` to collect files from the ``extracted_directory``. This is done by sending each filename through to the ``FileNameCollectorPlugin``, which in turn applies a named group regular expression (``named_regexp``) to the filename. The ``FileNameCollectorPlugin`` returns ``None`` if the given file should be skipped or it returns an object that must contain the ``student_id`` and ``file_id`` data, and can optionally contain the ``timestamp``, ``first_name``, ``last_name``, and ``email`` data.

Thus if using the default ``FileNameCollectorPlugin`` you must at least supply the ``student_id`` and ``file_id`` named groups. Of course this plugin assumes all extracted files have the same filename structure similar to downloaded notebook:

In [3]:
%%bash

ls -l downloaded/ps1/archive

total 32
-rw-rw-r-- 1 logan logan 18022 Jan 20 16:11 gradebook_xyz101_s1_2016_ps1_2016-02-10-17-30-10.zip
-rw-rw-r-- 1 logan logan  8830 Jan 20 16:46 ps1_hacker_attempt_2016-01-30-20-30-10_problem1.ipynb


With the ``nbgrader_config.py`` file created we can now run the ``nbgrader zip_collect`` sub-command, for example:

In [4]:
%%bash

nbgrader zip_collect ps1

[ZipCollectApp | INFO] Using file collector: FileNameCollectorPlugin
[ZipCollectApp | WARNING] Directory not found. Creating: /media/logan/External/Code_Dev/nbgrader/nbgrader/docs/source/user_guide/downloaded/ps1/extracted
[ZipCollectApp | INFO] Extracting file: /media/logan/External/Code_Dev/nbgrader/nbgrader/docs/source/user_guide/downloaded/ps1/archive/gradebook_xyz101_s1_2016_ps1_2016-02-10-17-30-10.zip
[ZipCollectApp | INFO] Copying file to: /media/logan/External/Code_Dev/nbgrader/nbgrader/docs/source/user_guide/downloaded/ps1/extracted/ps1_hacker_attempt_2016-01-30-20-30-10_problem1.ipynb
[ZipCollectApp | INFO] Start collecting files...
[ZipCollectApp | INFO] Processing file: /media/logan/External/Code_Dev/nbgrader/nbgrader/docs/source/user_guide/downloaded/ps1/extracted/ps1_bitdiddle_attempt_2016-01-30-15-30-10_jupyter.png
[ZipCollectApp | WARNING] Skipped. No match information provided.
[ZipCollectApp | INFO] Processing file: /media/logan/External/Code_Dev/nbgrader/nbgrader/doc

After running the ``nbgrader zip_collect`` sub-command, the archive (zip) files were extracted - and the non-archive files were copied - to the ``extracted_directory``:

In [5]:
%%bash

ls -l downloaded/ps1/extracted/

total 56
-rw-rw-r-- 1 logan logan 5733 Jan 20 16:47 ps1_bitdiddle_attempt_2016-01-30-15-30-10_jupyter.png
-rw-rw-r-- 1 logan logan 7712 Jan 20 16:47 ps1_bitdiddle_attempt_2016-01-30-15-30-10_problem1.ipynb
-rw-rw-r-- 1 logan logan 2228 Jan 20 16:47 ps1_bitdiddle_attempt_2016-01-30-15-30-10_problem2.ipynb
-rw-rw-r-- 1 logan logan 5733 Jan 20 16:47 ps1_hacker_attempt_2016-01-30-16-30-10_jupyter.png
-rw-rw-r-- 1 logan logan 8830 Jan 20 16:47 ps1_hacker_attempt_2016-01-30-16-30-10_myproblem1.ipynb
-rw-rw-r-- 1 logan logan 2358 Jan 20 16:47 ps1_hacker_attempt_2016-01-30-16-30-10_problem2.ipynb
-rw-rw-r-- 1 logan logan 8830 Jan 20 16:47 ps1_hacker_attempt_2016-01-30-20-30-10_problem1.ipynb


And these files were then collected and copied into the students ``submitted_directory``:

In [6]:
%%bash

ls -l submitted_zip

total 8
drwxrwxr-x 3 logan logan 4096 Jan 20 16:47 bitdiddle
drwxrwxr-x 3 logan logan 4096 Jan 20 16:47 hacker


In [7]:
%%bash

ls -l submitted_zip/hacker/ps1/

total 20
-rw-rw-r-- 1 logan logan 8830 Jan 20 16:47 problem1.ipynb
-rw-rw-r-- 1 logan logan 2358 Jan 20 16:47 problem2.ipynb
-rw-rw-r-- 1 logan logan   19 Jan 20 16:47 timestamp.txt


Unfortunately, for this demo, the timestamp strings from the filenames do not parse correctly:

In [8]:
%%bash

cat submitted_zip/hacker/ps1/timestamp.txt

2016-01-31 06:00:00

This is an issue with the underlying ``dateutils`` package used by ``nbgrader``. But not to worry, we can easily create a custom collector plugin to correct the timestamp strings when the files are collected:

In [9]:
%%file plugin.py

from nbgrader.plugins import FileNameCollectorPlugin


class CustomPlugin(FileNameCollectorPlugin):
    def collect(self, submission_file):
        info = super(CustomPlugin, self).collect(submission_file)
        if info is not None:
            info.timestamp = '{}-{}-{} {}:{}:{}'.format(
                *tuple(info.timestamp.split('-'))
            )
        return info

Writing plugin.py


In [10]:
%%bash

nbgrader zip_collect --force --collector=plugin.CustomPlugin ps1

[ZipCollectApp | INFO] Using file collector: CustomPlugin
[ZipCollectApp | WARNING] Clearing existing files in /media/logan/External/Code_Dev/nbgrader/nbgrader/docs/source/user_guide/downloaded/ps1/extracted
[ZipCollectApp | INFO] Extracting file: /media/logan/External/Code_Dev/nbgrader/nbgrader/docs/source/user_guide/downloaded/ps1/archive/gradebook_xyz101_s1_2016_ps1_2016-02-10-17-30-10.zip
[ZipCollectApp | INFO] Copying file to: /media/logan/External/Code_Dev/nbgrader/nbgrader/docs/source/user_guide/downloaded/ps1/extracted/ps1_hacker_attempt_2016-01-30-20-30-10_problem1.ipynb
[ZipCollectApp | INFO] Start collecting files...
[ZipCollectApp | INFO] Processing file: /media/logan/External/Code_Dev/nbgrader/nbgrader/docs/source/user_guide/downloaded/ps1/extracted/ps1_bitdiddle_attempt_2016-01-30-15-30-10_jupyter.png
[ZipCollectApp | WARNING] Skipped. No match information provided.
[ZipCollectApp | INFO] Processing file: /media/logan/External/Code_Dev/nbgrader/nbgrader/docs/source/user_g

The ``--force`` flag is used this time to overwrite existing extracted and submitted files. Now if we check the timestamp we see it parsed correctly:

In [11]:
%%bash

cat submitted_zip/hacker/ps1/timestamp.txt

2016-01-30 20:30:10

Note that there should only ever be *one* instructor who runs the ``nbgrader zip_collect`` command (and there should probably only be one instructor -- the same instructor -- who runs `nbgrader assign`, `nbgrader autograde` and `nbgrader formgrade` as well). However this does not mean that only one instructor can do the grading, it just means that only one instructor manages the assignment files. Other instructors can still perform grading by accessing the formgrader URL.